# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### LiH in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 12 

FCI energy: -7.882362286810939


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [5]:
H =lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 14 


Starting optimization:



Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 60 expectationvalues

active variables : 14

E=-7.86335762  angles= {(5, 1, 5, 1): 0.0, (2, 1, 5, 1): 0.0, (5, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (4, 1, 4, 1): 0.0, (2, 1, 2, 1): 0.0, (2, 0, 2, 0): 0.0, (3, 1, 3, 0): 0.0, (4, 1, 4, 0): 0.0, (3, 0, 3, 1): 0.0, (4, 0, 4, 1): 0.0, (3, 0, 3, 0): 0.0, (4, 0, 4, 0): 0.0, (5, 0, 5, 0): 0.0}  samples= None
E=-7.84198185  angles= {(5, 1, 5, 1): -0.24590444564819336, (2, 1, 5, 1): -0.09068489074707031, (5, 1, 2, 1): -0.024281024932861328, (3, 1, 3, 1): -0.04302263259887695, (4, 1, 4, 1): 0.015152454376220703, (2, 1, 2, 1): 0.015152454376220703, (2, 0, 2, 0): -0.0196380615234375, (3, 1, 3, 0): 0.0014743804931640625, (4, 1, 4, 0): 0.0014743804931640625, (3, 0, 3, 1): -0.00253295898437

E=-7.87942405  angles= {(5, 1, 5, 1): -0.10979637927012136, (2, 1, 5, 1): -0.04234170324223751, (5, 1, 2, 1): -0.0341512778141078, (3, 1, 3, 1): -0.004056200799151398, (4, 1, 4, 1): 0.0028934253513407705, (2, 1, 2, 1): 0.0028586337556640177, (2, 0, 2, 0): -0.0018347947270500584, (3, 1, 3, 0): 0.0003983062396873781, (4, 1, 4, 0): 0.0001211106937304057, (3, 0, 3, 1): -0.00015745056711499432, (4, 0, 4, 1): -0.0002932939364492596, (3, 0, 3, 0): -0.0005105142835432571, (4, 0, 4, 0): 5.398660866123641e-05, (5, 0, 5, 0): 0.00025568102714585296}  samples= None
E=-7.87942432  angles= {(5, 1, 5, 1): -0.10965314291096968, (2, 1, 5, 1): -0.04272835151040151, (5, 1, 2, 1): -0.034515326025702855, (3, 1, 3, 1): -0.004062283412117804, (4, 1, 4, 1): 0.002933796225638775, (2, 1, 2, 1): 0.002918811639181317, (2, 0, 2, 0): -0.0017904322035121147, (3, 1, 3, 0): 0.00041121313311492556, (4, 1, 4, 0): 0.00015723795409417922, (3, 0, 3, 1): -0.00022576656529261634, (4, 0, 4, 1): -0.000317035898299086, (3, 0, 3,

E=-7.87942438  angles= {(5, 1, 5, 1): -0.10967617261045227, (2, 1, 5, 1): -0.042668316474205055, (5, 1, 2, 1): -0.03445465926873494, (3, 1, 3, 1): -0.00406360079715535, (4, 1, 4, 1): 0.002909192897450759, (2, 1, 2, 1): 0.0029012173798825345, (2, 0, 2, 0): -0.0017967939925565785, (3, 1, 3, 0): 0.00042512775339349426, (4, 1, 4, 0): 0.00017013885616496595, (3, 0, 3, 1): -0.00021625982471557868, (4, 0, 4, 1): -0.0003290301121683644, (3, 0, 3, 0): -0.0005893389539833252, (4, 0, 4, 0): 0.00021118138082174906, (5, 0, 5, 0): 0.00037332522102213254}  samples= None
E=-7.87942438  angles= {(5, 1, 5, 1): -0.10967617415135623, (2, 1, 5, 1): -0.0426681962620055, (5, 1, 2, 1): -0.03445457535848804, (3, 1, 3, 1): -0.004063597488916166, (4, 1, 4, 1): 0.002909301029706838, (2, 1, 2, 1): 0.0029011448886366795, (2, 0, 2, 0): -0.0017968210185133255, (3, 1, 3, 0): 0.00042522438407488895, (4, 1, 4, 0): 0.00017015938139865088, (3, 0, 3, 1): -0.00021620332773850375, (4, 0, 4, 1): -0.0003290163564062779, (3, 0,

E=-7.87942438  angles= {(5, 1, 5, 1): -0.10967617559306789, (2, 1, 5, 1): -0.042668157066839504, (5, 1, 2, 1): -0.03445454720447028, (3, 1, 3, 1): -0.004063597279977245, (4, 1, 4, 1): 0.0029093077353409633, (2, 1, 2, 1): 0.0029011395359140628, (2, 0, 2, 0): -0.0017968158472724026, (3, 1, 3, 0): 0.00042524407938363966, (4, 1, 4, 0): 0.0001701627447909736, (3, 0, 3, 1): -0.00021619549455858416, (4, 0, 4, 1): -0.00032901237142144915, (3, 0, 3, 0): -0.0005892651672795374, (4, 0, 4, 0): 0.00021111324648822772, (5, 0, 5, 0): 0.00037323837482854106}  samples= None
E=-7.87942438  angles= {(5, 1, 5, 1): -0.10967617559153527, (2, 1, 5, 1): -0.04266815709689139, (5, 1, 2, 1): -0.03445454723387633, (3, 1, 3, 1): -0.004063597280098807, (4, 1, 4, 1): 0.00290930772976926, (2, 1, 2, 1): 0.0029011395402873084, (2, 0, 2, 0): -0.0017968158517773362, (3, 1, 3, 0): 0.00042524406268241733, (4, 1, 4, 0): 0.00017016274208989988, (3, 0, 3, 1): -0.00021619550115216637, (4, 0, 4, 1): -0.00032901237489128566, (3,

E=-7.87942438  angles= {(5, 1, 5, 1): -0.10967617559144091, (2, 1, 5, 1): -0.042668157098741444, (5, 1, 2, 1): -0.03445454723568663, (3, 1, 3, 1): -0.004063597280106292, (4, 1, 4, 1): 0.0029093077294262545, (2, 1, 2, 1): 0.002901139540556535, (2, 0, 2, 0): -0.0017968158520546694, (3, 1, 3, 0): 0.0004252440616542548, (4, 1, 4, 0): 0.0001701627419236161, (3, 0, 3, 1): -0.00021619550155808123, (4, 0, 4, 1): -0.0003290123751048961, (3, 0, 3, 0): -0.0005892651729941473, (4, 0, 4, 0): 0.00021111326214954642, (5, 0, 5, 0): 0.0003732383803686067}  samples= None
         Current function value: -7.879424
         Iterations: 23
         Function evaluations: 61
         Gradient evaluations: 49

Obtained UCCSD energy: -7.879424384684654


## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### LiH in STO-3G basis

Below we perform the entangler screening protocol for H2 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [9]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 631 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.123
2 : 0.0335
3 : 0.0335
4 : 0.024
5 : 0.024
6 : 0.0215
7 : 0.0121
8 : 0.0098
9 : 0.0098
10 : 0.0076
11 : 0.0076
12 : 0.0076
13 : 0.0076
14 : 0.0072
15 : 0.002
16 : 0.002
17 : 0.0013
18 : 0.0013
19 : 0.0012
20 : 0.0012
21 : 0.0007
22 : 0.0007
23 : 0.0004
24 : 0.0004
25 : 0.0002
26 : 0.0002
27 : 0.0
28 : 0.0
29 : 0.0
30 : 0.0
31 : 0.0
32 : 0.0

Selected entanglers:
1.0 [X2 Y3 X10 X11]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [10]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 50 expectationvalues

active variables : 25

E=-6.25839976  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.0}  samples= None
Optimization terminated successfully.
         Current function value: -6.258400
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1

Obtained QCC energy (1 entanglers): -6.2583997556447